## Azure Machine Learning での実験

学習スクリプトの変更箇所
- データセット (引数で引き渡す)
    - `as_mount()` : ファイルをマウントする
    - `as_download()` : ファイルをダウンロードする
- メトリック記録
    - `run.log(xxx)` で実験のメトリックとして記録

In [ ]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.widgets import RunDetails

In [ ]:
ws = Workspace.from_config()

In [ ]:
dataset = Dataset.get_by_name(ws, name='cifar10')

In [ ]:
experiment = Experiment(workspace=ws, name='3-aml-run')

### 学習スクリプトの実行設定

In [ ]:

config = ScriptRunConfig(source_directory='./code/pytorch-cloud',
                         script='train.py',
                         compute_target='cpu-cluster',
                         arguments=[
                            '--data_path', dataset.as_named_input('input').as_download(),
                            '--learning_rate', 0.003,
                            '--momentum', 0.92])



### Environment の呼び出し

In [ ]:
env = Environment.get(ws, "pytorch-env")

In [ ]:
config.run_config.environment = env

### 実行

In [ ]:
run = experiment.submit(config)
RunDetails(run).show()